<a href="https://colab.research.google.com/github/lsmanoel/PowerEletronics/blob/master/driver/driver_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de Circuito de Comando
Lucas Seara Manoel e Diogo Tavares


![alt text](https://raw.githubusercontent.com/lsmanoel/PowerEletronics/master/driver/images/buck_circuit.PNG)

### Para montar o circuito de controle foram escolhidos 2 modelos de CI's para a aplicação.

#### Um deles é o [MIC38C42/3/4/5](http://ww1.microchip.com/downloads/en/DeviceDoc/mic38C42.pdf) da Microship, encontrado no site da [Digikey](https://www.digikey.com/products/en/integrated-circuits-ics/pmic-voltage-regulators-dc-dc-switching-controllers/715?k=power%20supply%20driver&newproducts=1), possui PWM. Não há necessidade de isolamento para o conversor Buck, mas este CI apresenta possibilidade de isolamento.

### A seguir são apresentadas algumas características e um circuito de aplicação com isolamento sugerida pelo fabricante:

